In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatLogSlider, FloatSlider, interact


In [2]:

def make_NMR_signal(peaktops, t):
    """
    ピーク位置と時間軸を基にNMR信号を生成します。
    
    Parameters:
    - peaktops (ndarray): ピークの周波数位置の配列。
    - t (ndarray): 時間ポイントの配列。
    
    Returns:
    - signal (ndarray): 生成されたNMR信号。
    """

    signal_cos = np.zeros(len(t))
    signal_sin = np.zeros(len(t))

    # Calculate cos
    for peak in peaktops:
        signal_cos += np.cos(2 * np.pi * peak * t)
        signal_sin += np.sin(2 * np.pi * peak * t)

    # Add exponential decay
    signal_cos *= np.exp(-0.1 * t) 
    signal_sin *= np.exp(-0.1 * t) 
    return (signal_cos, signal_sin)


In [3]:
def add_gaussian_noise(signals, intensity=0.1):
    """
    信号にガウスノイズを追加します。
    パラメータ:
    - signal (ndarray): ノイズを追加する信号。
    - intensity (float): ノイズの強度。
    戻り値:
    - signal_noisy (ndarray): ノイズが追加された信号。
    """

    signal_noisy = signals + np.random.normal(0, intensity, len(signals[0]))
    return signal_noisy


In [4]:
def heterodyne(signals, reference, t):
    """
    ヘテロダイン検出を実行します。
    パラメータ:
    - signal (ndarray): 信号。
    - ref_signal (ndarray): 参照信号。
    戻り値:
    - signal_heterodyne (ndarray): ヘテロダイン信号。
    """

    het_cos = signals[0] * np.cos(2 * np.pi * reference * t)
    het_sin = signals[1] * np.sin(2 * np.pi * reference * t)
    return het_cos, het_sin 


In [7]:
def make_NMR_chart(range_max, N, peaktops, reference):
    """
    NMRチャートを生成します。
    パラメータ:
    - range_max (float): 最大周波数範囲。
    - N (int): データポイントの数。
    - peaktops (ndarray): ピークの周波数。
    - reference (float): 参照周波数。
    戻り値:
    - freq (ndarray): 周波数。
    - signal_positive (ndarray): NMR信号。
    """
    measure_time = N / range_max / 2
    t = np.linspace(0, measure_time, N, endpoint=False)
    signal = make_NMR_signal(peaktops, t)
    signal = add_gaussian_noise(signal)

    # ヘテロダイン混合の実行
    heterodyne_signal = heterodyne(signal, reference, t)

    # 高速フーリエ変換の実行
    FFTsignal =  np.real(np.fft.fft(heterodyne_signal[0]))[:N//2]+np.real(np.fft.fft(heterodyne_signal[1]))[:N//2]
    freq = np.fft.fftfreq(N, 1 / range_max / 2)[: N // 2]

    FFTsignal -= np.min(FFTsignal)
    FFTsignal /= np.max(FFTsignal)
    return freq, FFTsignal


In [ ]:
def plot_NMR_chart(N, peakshift, J1, J2):
    """
    NMRチャートをプロットします。
    パラメータ:
    - N (float): データポイントの数。
    - peakshift (float): ピークのシフト (Hz)。
    - J1 (float): 第一結合定数。
    - J2 (float): 第二結合定数。
    戻り値:
    - None
    """
    N = int(N)
    resonancefreq = 4* 10**6 # Hz

    # ピークトップの位置を取得
    peaktops = (
        np.array([(-J1 - J2) / 2, (-J1 + J2) / 2, (J1 - J2) / 2, (J1 + J2) / 2])
        + peakshift * resonancefreq/10**6
        + resonancefreq
    )

    # NMRチャートを描画
    
    x, y = make_NMR_chart(10*resonancefreq/10**6, N, peaktops, resonancefreq)
    
    # 化学シフト（ppm）に変換するために、周波数を参照周波数で割ります
    plt.plot(x/resonancefreq*10**6, y)
    plt.gca().invert_xaxis()#x軸を反転
    plt.show()

interact(
    plot_NMR_chart,
    N=FloatLogSlider(value=2**11, min=5, max=20, base=2, step=1),
    peakshift=FloatSlider(value=5, min=0, max=10, step=0.1),
    J1=FloatSlider(value=1.2, min=0, max=10, step=0.1),
    J2=FloatSlider(value=1.2, min=0, max=10, step=0.1),
)
pass

interactive(children=(FloatLogSlider(value=2048.0, base=2.0, description='N', max=20.0, min=5.0, step=1.0), Fl…